# Import

In [48]:
import pandas as pd 
from urllib.request import urlopen
from dotenv import dotenv_values
from sqlalchemy import create_engine, types
from sqlalchemy.dialects.postgresql import JSON as postgres_json
import pandas as pd
import requests
import json
from datetime import datetime
import time
import pprint

# Access API

In [49]:
years = [2023,2024,2025]
for i in years:
    if i == years[0]:
        response = urlopen(f'https://api.openf1.org/v1/sessions?session_name=Race&year={i}')
        data = json.loads(response.read().decode('utf-8'))
        df_sessions = pd.DataFrame(data)
    else:
        response = urlopen(f'https://api.openf1.org/v1/sessions?session_name=Race&year={i}')
        data = json.loads(response.read().decode('utf-8'))
        df = pd.DataFrame(data)
        df_sessions = pd.concat([df_sessions,df])

In [50]:
df_sessions

,meeting_key,session_key,location,date_start,date_end,session_type,session_name,country_key,country_code,country_name,circuit_key,circuit_short_name,gmt_offset,year
0,1141,7953,Sakhir,2023-03-05T15:00:00+00:00,2023-03-05T17:00:00+00:00,Race,Race,36,BRN,Bahrain,63,Sakhir,03:00:00,2023
1,1142,7779,Jeddah,2023-03-19T17:00:00+00:00,2023-03-19T19:00:00+00:00,Race,Race,153,KSA,Saudi Arabia,149,Jeddah,03:00:00,2023
2,1143,7787,Melbourne,2023-04-02T05:00:00+00:00,2023-04-02T07:00:00+00:00,Race,Race,5,AUS,Australia,10,Melbourne,10:00:00,2023
3,1207,9070,Baku,2023-04-30T11:00:00+00:00,2023-04-30T13:00:00+00:00,Race,Race,30,AZE,Azerbaijan,144,Baku,04:00:00,2023
4,1208,9078,Miami,2023-05-07T19:30:00+00:00,2023-05-07T21:30:00+00:00,Race,Race,19,USA,United States,151,Miami,-04:00:00,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1270,9896,Marina Bay,2025-10-05T12:00:00+00:00,2025-10-05T14:00:00+00:00,Race,Race,157,SGP,Singapore,61,Singapore,08:00:00,2025
18,1271,9888,Austin,2025-10-19T19:00:00+00:00,2025-10-19T21:00:00+00:00,Race,Race,19,USA,United States,9,Austin,-05:00:00,2025
19,1272,9877,Mexico City,2025-10-26T20:00:00+00:00,2025-10-26T22:00:00+00:00,Race,Race,8,MEX,Mexico,65,Mexico City,-06:00:00,2025
20,1273,9869,São Paulo,2025-11-09T17:00:00+00:00,2025-11-09T19:00:00+00:00,Race,Race,10,BRA,Brazil,14,Interlagos,-03:00:00,2025


# Create Key List

In [51]:
df_session_keys = df_sessions.drop(columns=['meeting_key', 'location', 'date_start', 'date_end','session_type', 'session_name', 'country_key', 'country_code','country_name', 'circuit_key', 'circuit_short_name', 'gmt_offset',
       'year'])

In [52]:
df_session_keys.to_csv('session_keys.csv', index=False)

# Upload raw_sessions to PSQL

In [53]:
df_sessions

,meeting_key,session_key,location,date_start,date_end,session_type,session_name,country_key,country_code,country_name,circuit_key,circuit_short_name,gmt_offset,year
0,1141,7953,Sakhir,2023-03-05T15:00:00+00:00,2023-03-05T17:00:00+00:00,Race,Race,36,BRN,Bahrain,63,Sakhir,03:00:00,2023
1,1142,7779,Jeddah,2023-03-19T17:00:00+00:00,2023-03-19T19:00:00+00:00,Race,Race,153,KSA,Saudi Arabia,149,Jeddah,03:00:00,2023
2,1143,7787,Melbourne,2023-04-02T05:00:00+00:00,2023-04-02T07:00:00+00:00,Race,Race,5,AUS,Australia,10,Melbourne,10:00:00,2023
3,1207,9070,Baku,2023-04-30T11:00:00+00:00,2023-04-30T13:00:00+00:00,Race,Race,30,AZE,Azerbaijan,144,Baku,04:00:00,2023
4,1208,9078,Miami,2023-05-07T19:30:00+00:00,2023-05-07T21:30:00+00:00,Race,Race,19,USA,United States,151,Miami,-04:00:00,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,1270,9896,Marina Bay,2025-10-05T12:00:00+00:00,2025-10-05T14:00:00+00:00,Race,Race,157,SGP,Singapore,61,Singapore,08:00:00,2025
18,1271,9888,Austin,2025-10-19T19:00:00+00:00,2025-10-19T21:00:00+00:00,Race,Race,19,USA,United States,9,Austin,-05:00:00,2025
19,1272,9877,Mexico City,2025-10-26T20:00:00+00:00,2025-10-26T22:00:00+00:00,Race,Race,8,MEX,Mexico,65,Mexico City,-06:00:00,2025
20,1273,9869,São Paulo,2025-11-09T17:00:00+00:00,2025-11-09T19:00:00+00:00,Race,Race,10,BRA,Brazil,14,Interlagos,-03:00:00,2025


In [54]:
df_sessions.columns

Index(['meeting_key', 'session_key', 'location', 'date_start', 'date_end',
       'session_type', 'session_name', 'country_key', 'country_code',
       'country_name', 'circuit_key', 'circuit_short_name', 'gmt_offset',
       'year'],
      dtype='object')

In [55]:
config = dotenv_values()

pg_user = config['POSTGRES_USER']
pg_pass = config['POSTGRES_PASS']
pg_host = config['POSTGRES_HOST']
pg_port = config['POSTGRES_PORT']
pg_db = config['POSTGRES_DB']
pg_schema = config['POSTGRES_SCHEMA']

url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'
engine = create_engine(url, echo=False)

In [56]:
# defining data types for the DB
dtype_dict = {
    'meeting_key': types.Integer,
    'session_key': types.Integer,
    'location': types.String,
    'date_start': types.DateTime,
    'date_end': types.DateTime,
    'session_type': types.String,
    'session_name': types.String,
    'country_key': types.Integer,
    'country_code': types.String,
    'country_name': types.String,
    'circuit_key': types.Integer,
    'circuit_short_name': types.String,
    'gmt_offset': types.String,        # could be Interval if stored as "+02:00"
    'year': types.Integer
}

In [57]:
df_sessions.to_sql(name = 'raw_sessions', 
                       con = engine, 
                       schema = pg_schema, 
                       if_exists='replace', 
                       dtype=dtype_dict,
                       index=False
                      )

68